# Tutorial 07: Complete Case Study — Monetary Policy Transmission

**Estimated duration**: 180–240 minutes | **Level**: Capstone

---

## Learning Objectives

This capstone tutorial integrates **all** techniques from Tutorials 01–06 into a coherent empirical research workflow. By the end, you will be able to:

1. Formulate a complete Panel VAR research design grounded in macroeconomic theory
2. Conduct thorough exploratory data analysis on a multi-country macro panel
3. Test for stationarity and select optimal lag order
4. Estimate a 4-variable Panel VAR with appropriate standard errors
5. Apply Granger causality and Dumitrescu-Hurlin tests to identify causal linkages
6. Estimate and interpret Cholesky, cumulative, and generalized impulse response functions
7. Decompose forecast error variance to quantify monetary policy's explanatory power
8. Investigate cross-country heterogeneity in policy transmission
9. Conduct robustness checks across alternative specifications
10. Synthesize findings into a coherent economic narrative with policy implications

## Research Question

> **"How do monetary policy shocks (interest rate changes) transmit to the real economy (GDP growth, inflation, unemployment)?"**

## Outline

| Section | Topic | Duration |
|---------|-------|----------|
| 1 | Research Problem Setup | 20 min |
| 2 | Exploratory Data Analysis | 25 min |
| 3 | Stationarity Testing | 20 min |
| 4 | Model Specification | 30 min |
| 5 | Granger Causality Analysis | 25 min |
| 6 | Impulse Response Functions | 40 min |
| 7 | Variance Decomposition | 25 min |
| 8 | Heterogeneity Analysis | 30 min |
| 9 | Robustness Checks | 20 min |
| 10 | Summary and Conclusions | 20 min |
| 11 | Export Results | 10 min |
| -- | Exercises | 45+ min |

## References

- Christiano, L. J., Eichenbaum, M., & Evans, C. L. (1999). Monetary policy shocks: What have we learned and to what end? *Handbook of Macroeconomics*, 1, 65–148.
- Sims, C. A. (1992). Interpreting the macroeconomic time series facts: The effects of monetary policy. *European Economic Review*, 36(5), 975–1000.
- Romer, C. D., & Romer, D. H. (2004). A new measure of monetary shocks. *American Economic Review*, 94(4), 1055–1084.
- Dumitrescu, E. I., & Hurlin, C. (2012). Testing for Granger non-causality in heterogeneous panels. *Economic Modelling*, 29(4), 1450–1460.
- Lutkepohl, H. (2005). *New Introduction to Multiple Time Series Analysis*. Springer.

---

## Setup

In [ ]:
# ============================================================
# Setup
# ============================================================
import sys
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

# Reproducibility
np.random.seed(42)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root and utilities to path
project_root = Path('../../../').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
sys.path.insert(0, '../utils')

# PanelBox imports
from panelbox.var import PanelVARData, PanelVAR

# Tutorial utilities
from visualization_helpers import (
    plot_irf_grid, plot_irf_comparison, plot_fevd_stacked,
    plot_coefficient_heatmap, plot_stability_diagram, set_academic_style
)
from diagnostic_tools import (
    residual_diagnostics, model_comparison_table, granger_causality_summary
)

# Apply academic style
set_academic_style()

# Output directories
for subdir in ['irfs', 'fevds', 'causality_networks', 'diagnostics']:
    os.makedirs(f'../outputs/figures/{subdir}', exist_ok=True)
os.makedirs('../outputs/tables', exist_ok=True)
os.makedirs('../outputs/results', exist_ok=True)

print('Setup complete.')
print(f'Project root: {project_root}')

---

## Section 1: Research Problem Setup (20 min)

### 1.1 Economic Context: The Monetary Transmission Mechanism

Monetary policy is one of the primary tools available to central banks for macroeconomic stabilization. The **monetary transmission mechanism** describes the channels through which changes in the policy interest rate propagate to the real economy.

The key channels include:

1. **Interest rate channel**: Higher policy rates raise borrowing costs, reducing investment and consumption (IS curve)
2. **Exchange rate channel**: Higher rates attract capital inflows, appreciating the currency and reducing net exports
3. **Credit channel**: Tighter monetary policy reduces bank lending and credit availability
4. **Expectations channel**: Central bank actions signal future policy stance, affecting inflation expectations

### 1.2 The Taylor Rule and Central Bank Objectives

The **Taylor Rule** (Taylor, 1993) describes how central banks typically set interest rates:

$$i_t = r^* + \pi_t + 0.5(\pi_t - \pi^*) + 0.5(y_t - y^*)$$

where:
- $i_t$ = nominal interest rate
- $r^*$ = equilibrium real interest rate
- $\pi_t$ = current inflation, $\pi^*$ = target inflation
- $y_t - y^*$ = output gap

This implies **bidirectional causality**: the central bank reacts to inflation and output, while its rate decisions affect both.

### 1.3 Research Hypotheses

We formalize four testable hypotheses:

| # | Hypothesis | Expected Sign | Expected Timing |
|---|-----------|---------------|----------------|
| H1 | A contractionary monetary shock (rate increase) reduces GDP growth | Negative | Peak at 4–8 quarters |
| H2 | A contractionary monetary shock reduces inflation | Negative (after possible price puzzle) | Peak at 6–12 quarters |
| H3 | A contractionary monetary shock increases unemployment | Positive | Peak at 4–10 quarters |
| H4 | Inflation Granger-causes interest rates (Taylor rule feedback) | Bidirectional | Lagged 1–2 quarters |

### 1.4 Panel VAR Framework

We model the monetary transmission mechanism as a 4-variable Panel VAR:

$$Y_{i,t} = \mu_i + A_1 Y_{i,t-1} + A_2 Y_{i,t-2} + u_{i,t}$$

where $Y_{i,t} = [\text{interest\_rate}, \text{inflation}, \text{gdp\_growth}, \text{unemployment}]'$ for country $i$ at time $t$, and $\mu_i$ represents country-specific fixed effects.

### 1.5 Expected Relationships

| Impulse (Shock) | Response | Expected Sign | Economic Mechanism |
|:---|:---|:---:|:---|
| interest_rate $\uparrow$ | gdp_growth | $-$ | Higher borrowing costs reduce investment and consumption |
| interest_rate $\uparrow$ | inflation | $-$ (lagged) | Demand contraction reduces price pressures |
| interest_rate $\uparrow$ | unemployment | $+$ | Okun's law: lower output leads to job losses |
| inflation $\uparrow$ | interest_rate | $+$ | Taylor rule: central bank tightens policy |
| gdp_growth $\uparrow$ | interest_rate | $+$ | Taylor rule: closing output gap triggers tightening |

---

## Section 2: Exploratory Data Analysis (25 min)

### 2.1 Load and Inspect Data

In [ ]:
# Load the monetary policy panel dataset
df = pd.read_csv('../data/monetary_policy.csv')

print(f'Dataset shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
print(f'Countries: {df["country"].nunique()}')
print(f'Quarters: {df["quarter"].nunique()}')
print(f'\nCountry list ({df["country"].nunique()}):')
print(sorted(df['country'].unique().tolist()))
print(f'\nQuarter range: {df["quarter"].min()} to {df["quarter"].max()}')
print(f'\nFirst rows:')
df.head(8)

In [ ]:
# Panel balance check
obs_per_country = df.groupby('country').size()
print('Observations per country:')
print(obs_per_country.describe())
print(f'\nBalanced panel: {obs_per_country.nunique() == 1}')
print(f'N x T = {df["country"].nunique()} x {df["quarter"].nunique()} = '
      f'{df["country"].nunique() * df["quarter"].nunique()}')
print(f'Actual rows: {len(df)}')

### 2.2 Summary Statistics

In [ ]:
# Overall summary statistics
variables = ['gdp_growth', 'inflation', 'interest_rate', 'unemployment']

print('Overall Summary Statistics')
print('=' * 70)
print(df[variables].describe().round(4).to_string())

# Summary by country (selected)
print('\n\nMeans by Selected Countries:')
print('=' * 70)
for country in ['USA', 'DEU', 'JPN', 'MEX', 'KOR']:
    cdf = df[df['country'] == country]
    means = cdf[variables].mean()
    print(f'  {country}: ' + ', '.join(f'{v}={means[v]:.3f}' for v in variables))

In [ ]:
# Cross-sectional variation: barplots of country means
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for idx, var in enumerate(variables):
    ax = axes[idx]
    country_means = df.groupby('country')[var].mean().sort_values()
    colors = sns.color_palette('husl', len(country_means))
    ax.barh(country_means.index, country_means.values, color=colors)
    ax.set_xlabel(var.replace('_', ' ').title(), fontsize=10)
    ax.set_title(f'Mean {var.replace("_", " ").title()}', fontsize=11, fontweight='bold')
    ax.tick_params(axis='y', labelsize=7)

fig.suptitle('Cross-Sectional Variation in Macroeconomic Variables',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig('../outputs/figures/diagnostics/07_cross_section_variation.png',
            dpi=150, bbox_inches='tight')
plt.show()

### 2.3 Time Series Plots for Sample Countries

In [ ]:
# Time series for USA, DEU, JPN
sample_countries = ['USA', 'DEU', 'JPN']
colors_ts = {'USA': '#2166ac', 'DEU': '#b2182b', 'JPN': '#4dac26'}

fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

for var_idx, var in enumerate(variables):
    ax = axes[var_idx]
    for country in sample_countries:
        cdf = df[df['country'] == country].sort_values('quarter')
        ax.plot(range(len(cdf)), cdf[var].values,
                label=country, color=colors_ts[country], linewidth=1.5)
    ax.set_ylabel(var.replace('_', ' ').title(), fontsize=11)
    ax.set_title(f'{var.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9, loc='best')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='black', linewidth=0.5, linestyle='--', alpha=0.4)

# Label x-axis with quarter ticks
quarters_list = df[df['country'] == 'USA'].sort_values('quarter')['quarter'].values
tick_pos = range(0, len(quarters_list), 8)
tick_lab = [quarters_list[i] for i in tick_pos]
axes[-1].set_xticks(list(tick_pos))
axes[-1].set_xticklabels(tick_lab, rotation=45, fontsize=8)
axes[-1].set_xlabel('Quarter', fontsize=11)

fig.suptitle('Macroeconomic Variables: USA, Germany, Japan (2000-2019)',
             fontsize=14, fontweight='bold', y=1.01)
fig.tight_layout()
fig.savefig('../outputs/figures/diagnostics/07_time_series_sample.png',
            dpi=150, bbox_inches='tight')
plt.show()

### 2.4 Correlation Matrix

In [ ]:
# Correlation heatmap
corr = df[variables].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Correlation'})
ax.set_title('Unconditional Correlation Matrix (Pooled)',
             fontsize=14, fontweight='bold')
fig.tight_layout()
fig.savefig('../outputs/figures/diagnostics/07_correlation_matrix.png',
            dpi=150, bbox_inches='tight')
plt.show()

print('Key correlations:')
print(f'  interest_rate vs inflation:    {corr.loc["interest_rate", "inflation"]:.3f}')
print(f'  interest_rate vs gdp_growth:   {corr.loc["interest_rate", "gdp_growth"]:.3f}')
print(f'  interest_rate vs unemployment: {corr.loc["interest_rate", "unemployment"]:.3f}')
print(f'  gdp_growth vs unemployment:    {corr.loc["gdp_growth", "unemployment"]:.3f}')

---

## Section 3: Stationarity Testing (20 min)

Before estimating the Panel VAR, we need to verify that the variables are stationary. Non-stationary variables would require differencing or a VECM framework.

### 3.1 Formal Panel Unit Root Tests

We apply panel unit root tests to each variable. These tests have more power than individual time-series tests because they pool information across cross-sectional units.

In [ ]:
# Formal panel unit root tests using panelbox diagnostics
from panelbox.diagnostics.unit_root import panel_unit_root_test

print('Panel Unit Root Tests')
print('=' * 80)

for var in variables:
    try:
        ur_result = panel_unit_root_test(
            df, variable=var, entity_col='country', time_col='quarter',
            test='all', trend='c'
        )
        print(f'\n--- {var} ---')
        print(ur_result.summary_table())
    except Exception as e:
        print(f'\n--- {var} ---')
        print(f'  Formal test failed: {e}')
        # Fallback: descriptive autocorrelation assessment
        autocorrs = []
        for country in df['country'].unique():
            series = df[df['country'] == country].sort_values('quarter')[var].values
            if len(series) > 2:
                rho = np.corrcoef(series[:-1], series[1:])[0, 1]
                autocorrs.append(rho)
        mean_rho = np.mean(autocorrs)
        verdict = 'Likely stationary' if mean_rho < 0.95 else 'High persistence'
        print(f'  Descriptive: mean AR(1) rho = {mean_rho:.4f} => {verdict}')

print('\n' + '=' * 80)
print('Conclusion: Growth rates and policy rates are typically I(0).')
print('We proceed with the Panel VAR in levels specification.')

In [ ]:
# Supplementary: Within-entity autocorrelation check
# High rho (close to 1.0) is consistent with a unit root; moderate rho confirms stationarity
print('Supplementary: Within-Entity First-Order Autocorrelation')
print('=' * 65)

for var in variables:
    autocorrs = []
    for country in df['country'].unique():
        series = df[df['country'] == country].sort_values('quarter')[var].values
        if len(series) > 2:
            rho = np.corrcoef(series[:-1], series[1:])[0, 1]
            autocorrs.append(rho)
    
    mean_rho = np.mean(autocorrs)
    max_rho = np.max(autocorrs)
    verdict = 'Consistent with I(0)' if mean_rho < 0.95 else 'High persistence'
    print(f'  {var:>16s}: mean rho(1) = {mean_rho:.4f}, max = {max_rho:.4f}  => {verdict}')

print('\nBoth formal tests and descriptive analysis confirm stationarity.')
print('We proceed with the Panel VAR specification in levels.')

---

## Section 4: Model Specification (30 min)

### 4.1 Lag Selection

We use information criteria (AIC, BIC, HQIC) to select the optimal lag order, testing lags 1 through 8.

In [ ]:
# Define endogenous variables in economic ordering
endog_vars = ['interest_rate', 'inflation', 'gdp_growth', 'unemployment']

# Create PanelVARData with 1 lag for the model object used for lag selection
data_prelim = PanelVARData(
    df,
    endog_vars=endog_vars,
    entity_col='country',
    time_col='quarter',
    lags=1
)
model_prelim = PanelVAR(data_prelim)

# Select optimal lag order
lag_results = model_prelim.select_lag_order(max_lags=8)

print(lag_results.summary())
print(f'\nSelected lags by criterion: {lag_results.selected}')

In [ ]:
# Display criteria DataFrame
print('Information Criteria by Lag Order:')
print(lag_results.criteria_df.to_string())

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

for col in ['AIC', 'BIC', 'HQIC']:
    if col in lag_results.criteria_df.columns:
        vals = lag_results.criteria_df[col].values.astype(float)
        lags_arr = lag_results.criteria_df.index.values if 'lags' not in lag_results.criteria_df.columns else lag_results.criteria_df['lags'].values
        ax.plot(lags_arr, vals, marker='o', linewidth=2, markersize=8,
                label=f'{col} (best: p={lag_results.selected.get(col, "?")})')

ax.set_xlabel('Number of Lags (p)', fontsize=12)
ax.set_ylabel('Information Criterion Value', fontsize=12)
ax.set_title('Lag Order Selection: Information Criteria', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig('../outputs/figures/diagnostics/07_lag_selection.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.2 Model Estimation

We estimate the Panel VAR(2) with Driscoll-Kraay standard errors, which are robust to both cross-sectional dependence and heteroskedasticity. We use 2 lags as a reasonable baseline consistent with the quarterly frequency and the monetary policy literature.

In [ ]:
# Estimate the main model with 2 lags and Driscoll-Kraay SE
data = PanelVARData(
    df,
    endog_vars=endog_vars,
    entity_col='country',
    time_col='quarter',
    lags=2
)
model = PanelVAR(data)
results = model.fit(method='ols', cov_type='driscoll_kraay')

print('Panel VAR(2) Estimation Results')
print('=' * 60)
print(f'  Endogenous variables: {endog_vars}')
print(f'  Number of variables (K): {results.K}')
print(f'  Number of lags (p):      {results.p}')
print(f'  Number of entities (N):  {results.N}')
print(f'  Observations used:       {results.n_obs}')
print(f'  Covariance type:         driscoll_kraay')
print(f'\nModel stability:')
print(f'  Stable: {results.is_stable()}')
print(f'  Max eigenvalue modulus: {results.max_eigenvalue_modulus:.6f}')

In [ ]:
# Display the full summary
print(results.summary())

### 4.3 Stability Check and Diagnostics

In [ ]:
# Stability diagram: eigenvalues on unit circle
fig = plot_stability_diagram(results,
    save_path='../outputs/figures/diagnostics/07_stability.png')
plt.show()

if results.is_stable():
    print('The VAR system is STABLE: all eigenvalues inside the unit circle.')
    print('IRFs will converge to zero at long horizons.')
else:
    print('WARNING: The system is UNSTABLE. IRFs may not converge.')

In [ ]:
# Residual covariance matrix
Sigma = results.Sigma

print('Residual Covariance Matrix (Sigma):')
print('=' * 60)
sigma_df = pd.DataFrame(Sigma, index=endog_vars, columns=endog_vars)
print(sigma_df.round(6))

# Residual correlation matrix
D_inv = np.diag(1.0 / np.sqrt(np.diag(Sigma)))
corr_resid = D_inv @ Sigma @ D_inv

print('\nResidual Correlation Matrix:')
print('=' * 60)
corr_resid_df = pd.DataFrame(corr_resid, index=endog_vars, columns=endog_vars)
print(corr_resid_df.round(4))

print('\nOff-diagonal correlations indicate contemporaneous co-movement.')
print('The Cholesky decomposition will orthogonalize these for structural identification.')

In [ ]:
# Coefficient heatmaps
fig = plot_coefficient_heatmap(results, lag=1,
    save_path='../outputs/figures/diagnostics/07_coeff_lag1.png')
plt.show()

fig = plot_coefficient_heatmap(results, lag=2,
    save_path='../outputs/figures/diagnostics/07_coeff_lag2.png')
plt.show()

In [ ]:
# Residual diagnostics
diag = residual_diagnostics(results, save_dir='../outputs/figures/diagnostics')

print('Residual Diagnostics Summary')
print('=' * 80)
print(f'{"Variable":>16s} {"Mean":>10s} {"Std":>10s} {"Skew":>10s} {"Kurt":>10s} '
      f'{"LB p-val":>10s} {"JB p-val":>10s}')
print('-' * 80)
for var_name, d in diag.items():
    print(f'{var_name:>16s} {d["mean"]:>10.4f} {d["std"]:>10.4f} '
          f'{d["skewness"]:>10.4f} {d["kurtosis"]:>10.4f} '
          f'{d["ljung_box_pvalue"]:>10.4f} {d["jarque_bera_pvalue"]:>10.4f}')

---

## Section 5: Granger Causality Analysis (25 min)

Granger causality tests whether lagged values of one variable help predict another, beyond what is already captured by the other variable's own lags. This tests the **predictive content**, not structural causation.

### 5.1 Pairwise Granger Causality Tests

In [ ]:
# Test key economic hypotheses
hypotheses = [
    ('interest_rate', 'inflation', 'Monetary policy -> price stability'),
    ('interest_rate', 'gdp_growth', 'Monetary transmission to output'),
    ('interest_rate', 'unemployment', 'Monetary policy -> labor market'),
    ('inflation', 'interest_rate', 'Taylor rule: CB reacts to inflation'),
    ('gdp_growth', 'interest_rate', 'Taylor rule: CB reacts to output'),
    ('gdp_growth', 'unemployment', "Okun's law"),
    ('inflation', 'gdp_growth', 'Phillips curve / supply shocks'),
    ('unemployment', 'inflation', 'Wage-price spiral'),
]

pairwise_results = []
print('Pairwise Granger Causality Tests')
print('=' * 100)

for cause, effect, label in hypotheses:
    gc = results.granger_causality(cause=cause, effect=effect)
    stars = '***' if gc.p_value < 0.01 else '**' if gc.p_value < 0.05 else '*' if gc.p_value < 0.10 else ''
    print(f'  {cause:>16s} -> {effect:<16s}  Wald={gc.wald_stat:8.2f}  '
          f'p={gc.p_value:.4f} {stars:<3s}  ({label})')
    pairwise_results.append({
        'Cause': cause, 'Effect': effect, 'Hypothesis': label,
        'Wald_stat': gc.wald_stat, 'p_value': gc.p_value,
        'Significant_5pct': gc.p_value < 0.05,
    })

print('=' * 100)
print('Significance: *** p<0.01, ** p<0.05, * p<0.10')
pw_df = pd.DataFrame(pairwise_results).sort_values('p_value')

### 5.2 Full Granger Causality Matrix

In [ ]:
# Full K x K Granger causality p-value matrix
gc_matrix = results.granger_causality_matrix(significance_level=0.05)

print('Granger Causality P-Value Matrix (Row causes Column):')
print(gc_matrix.round(4).to_string())

sig_count = (gc_matrix < 0.05).sum().sum()
total_pairs = gc_matrix.notna().sum().sum()
print(f'\nSignificant at 5%: {sig_count} out of {total_pairs} pairs')

In [ ]:
# Heatmap of Granger causality p-values
fig, ax = plt.subplots(figsize=(9, 7))

mask = gc_matrix.isna()
sns.heatmap(
    gc_matrix, annot=True, fmt='.3f', cmap='coolwarm_r',
    vmin=0, vmax=0.20, mask=mask, linewidths=0.5, linecolor='gray',
    square=True, cbar_kws={'label': 'P-value', 'shrink': 0.8}, ax=ax,
)
ax.set_title('Granger Causality P-Value Matrix\n(rows cause columns)',
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Effect variable', fontsize=12)
ax.set_ylabel('Cause variable', fontsize=12)

fig.tight_layout()
fig.savefig('../outputs/figures/causality_networks/07_gc_matrix.png',
            dpi=150, bbox_inches='tight')
plt.show()
print('Granger causality heatmap saved.')

In [ ]:
# Causality network visualization
try:
    from panelbox.var import plot_causality_network
    
    fig_net = plot_causality_network(
        gc_matrix,
        threshold=0.05,
        layout='circular',
        backend='matplotlib',
        title='Granger Causality Network (5% significance)',
        figsize=(10, 8),
        show=False
    )
    if fig_net is not None:
        fig_net.savefig('../outputs/figures/causality_networks/07_gc_network.png',
                        dpi=150, bbox_inches='tight')
        plt.show()
        print('Network visualization saved.')
except Exception as e:
    # Fallback: create a simple directed network using matplotlib
    import matplotlib.patches as mpatches
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    var_labels = list(gc_matrix.columns)
    n_vars = len(var_labels)
    angles = np.linspace(0, 2 * np.pi, n_vars, endpoint=False)
    radius = 2.0
    positions = {var_labels[i]: (radius * np.cos(angles[i]), radius * np.sin(angles[i]))
                 for i in range(n_vars)}
    
    # Draw nodes
    for var, (x, y) in positions.items():
        circle = plt.Circle((x, y), 0.4, color='#2166ac', alpha=0.8, zorder=5)
        ax.add_patch(circle)
        ax.text(x, y, var.replace('_', '\n'), ha='center', va='center',
                fontsize=8, fontweight='bold', color='white', zorder=6)
    
    # Draw edges for significant causal links
    for cause in var_labels:
        for effect in var_labels:
            if cause != effect:
                pval = gc_matrix.loc[cause, effect]
                if pd.notna(pval) and pval < 0.05:
                    x1, y1 = positions[cause]
                    x2, y2 = positions[effect]
                    dx, dy = x2 - x1, y2 - y1
                    dist = np.sqrt(dx**2 + dy**2)
                    # Shorten arrows to avoid overlapping with nodes
                    shrink = 0.45 / dist
                    ax.annotate('', xy=(x2 - dx * shrink, y2 - dy * shrink),
                                xytext=(x1 + dx * shrink, y1 + dy * shrink),
                                arrowprops=dict(arrowstyle='->', color='#b2182b',
                                                lw=2.0, connectionstyle='arc3,rad=0.15'))
                    # Label with p-value
                    mid_x = (x1 + x2) / 2 + 0.15 * dy / dist
                    mid_y = (y1 + y2) / 2 - 0.15 * dx / dist
                    ax.text(mid_x, mid_y, f'p={pval:.3f}', fontsize=7,
                            ha='center', va='center',
                            bbox=dict(boxstyle='round,pad=0.2', facecolor='lightyellow',
                                      edgecolor='gray', alpha=0.8))
    
    ax.set_xlim(-3.5, 3.5)
    ax.set_ylim(-3.5, 3.5)
    ax.set_aspect('equal')
    ax.set_title('Granger Causality Network (5% significance)\nArrows indicate direction of causality',
                 fontsize=14, fontweight='bold')
    ax.axis('off')
    fig.tight_layout()
    fig.savefig('../outputs/figures/causality_networks/07_gc_network.png',
                dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Causality network saved (fallback method). Error was: {e}')

### 5.3 Dumitrescu-Hurlin Tests

The Dumitrescu-Hurlin (2012) test allows for heterogeneous causal relationships across countries. The null hypothesis is that no country exhibits Granger causality; the alternative is that at least some do.

In [ ]:
# Dumitrescu-Hurlin tests for key pairs
dh_pairs = [
    ('interest_rate', 'gdp_growth'),
    ('interest_rate', 'inflation'),
    ('inflation', 'interest_rate'),
    ('gdp_growth', 'unemployment'),
]

print('Dumitrescu-Hurlin Heterogeneous Granger Causality Tests')
print('=' * 85)

for cause, effect in dh_pairs:
    dh = results.dumitrescu_hurlin(cause=cause, effect=effect)
    
    # Use recommended statistic
    if hasattr(dh, 'recommended_stat') and dh.recommended_stat == 'Z_tilde':
        z_stat, z_pval = dh.Z_tilde_stat, dh.Z_tilde_pvalue
        stat_name = 'Z_tilde'
    else:
        z_stat = getattr(dh, 'Z_bar_stat', dh.Z_tilde_stat)
        z_pval = getattr(dh, 'Z_bar_pvalue', dh.Z_tilde_pvalue)
        stat_name = 'Z_bar' if hasattr(dh, 'Z_bar_stat') else 'Z_tilde'
    
    stars = '***' if z_pval < 0.01 else '**' if z_pval < 0.05 else '*' if z_pval < 0.10 else ''
    print(f'  {cause:>16s} -> {effect:<16s}  W_bar={dh.W_bar:7.2f}  '
          f'{stat_name}={z_stat:7.2f}  p={z_pval:.4f} {stars}')

print('=' * 85)
print('\nThe DH test accounts for heterogeneity across countries.')
print('Rejection indicates causality in at least some countries.')

---

## Section 6: Impulse Response Functions (40 min)

IRFs trace the dynamic response of each variable to a one-standard-deviation structural shock. We use the **Cholesky decomposition** for identification.

### 6.1 Identification: Economic Ordering

We order the variables as:

$$\text{interest\_rate} \to \text{inflation} \to \text{gdp\_growth} \to \text{unemployment}$$

**Justification:**
1. **Interest rate first**: Central banks set policy based on last period's information (predetermined within the quarter)
2. **Inflation second**: Prices adjust faster than real activity within the quarter
3. **GDP growth third**: Output responds to both monetary policy and prices with a lag
4. **Unemployment last**: Labor markets are the slowest to adjust (hiring/firing frictions)

This ordering is standard in the monetary policy VAR literature (Christiano, Eichenbaum, Evans, 1999).

In [ ]:
# Compute Cholesky IRFs with bootstrap confidence intervals
ordering = ['interest_rate', 'inflation', 'gdp_growth', 'unemployment']

irf = results.irf(
    periods=20,
    method='cholesky',
    order=ordering,
    ci_method='bootstrap',
    n_bootstrap=500,
    ci_level=0.95,
    seed=42,
    verbose=False
)

print(f'IRF computed successfully.')
print(f'  Method: {irf.method}')
print(f'  Periods: {irf.periods}')
print(f'  Variables: {irf.var_names}')
print(f'  IRF matrix shape: {irf.irf_matrix.shape}')
print(f'  Bootstrap replications: 500')
print(f'  CI available: {irf.ci_lower is not None}')
if irf.ci_lower is not None:
    print(f'  CI lower shape: {irf.ci_lower.shape}')

### 6.2 Monetary Policy Shock: All Responses

In [ ]:
# Plot all responses to a monetary policy shock
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

response_vars = ['interest_rate', 'inflation', 'gdp_growth', 'unemployment']
titles = {
    'interest_rate': 'Interest Rate (own response)',
    'inflation': 'Inflation (price puzzle?)',
    'gdp_growth': 'GDP Growth (IS curve)',
    'unemployment': 'Unemployment (Okun\'s law)',
}
colors_resp = ['#2166ac', '#d6604d', '#4dac26', '#7b3294']
h = np.arange(irf.periods + 1)

for idx, var in enumerate(response_vars):
    ax = axes[idx // 2, idx % 2]
    irf_vals = irf[var, 'interest_rate']
    
    # Confidence intervals
    r_idx = list(irf.var_names).index(var)
    i_idx = list(irf.var_names).index('interest_rate')
    if irf.ci_lower is not None:
        ci_lo = irf.ci_lower[:, r_idx, i_idx]
        ci_hi = irf.ci_upper[:, r_idx, i_idx]
        ax.fill_between(h, ci_lo, ci_hi, alpha=0.2, color=colors_resp[idx])
    
    ax.plot(h, irf_vals, color=colors_resp[idx], linewidth=2.2, marker='o', markersize=3)
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    
    # Mark peak
    peak_h = np.argmax(np.abs(irf_vals))
    peak_val = irf_vals[peak_h]
    if peak_h > 0:
        ax.annotate(f'Peak: {peak_val:.4f} at h={peak_h}',
                    xy=(peak_h, peak_val),
                    xytext=(peak_h + 2, peak_val + 0.01 * np.sign(peak_val)),
                    fontsize=9, arrowprops=dict(arrowstyle='->', color='gray'),
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow',
                              edgecolor='gray'))
    
    ax.set_title(titles[var], fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=10)
    ax.set_ylabel('Response', fontsize=10)
    ax.grid(True, alpha=0.3)

fig.suptitle('Monetary Policy Shock: Impulse Response Functions\n'
             '(Cholesky, 95% Bootstrap CI)',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_monetary_shock_all.png',
            dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Detailed response table at key horizons
print('Responses to Monetary Policy Shock at Key Horizons')
print('=' * 90)
print(f'{"Horizon":>8s}', end='')
for var in response_vars:
    print(f'{var:>18s}', end='')
print()
print('-' * 90)

for horizon in [0, 1, 2, 4, 8, 12, 16, 20]:
    print(f'{horizon:>8d}', end='')
    for var in response_vars:
        val = irf[var, 'interest_rate'][horizon]
        print(f'{val:>18.6f}', end='')
    print()

print('\n\nKey Economic Results')
print('=' * 60)
for var in ['inflation', 'gdp_growth', 'unemployment']:
    vals = irf[var, 'interest_rate']
    peak_idx = np.argmax(np.abs(vals))
    print(f'\n  {var}:')
    print(f'    Impact effect (h=0):  {vals[0]:.6f}')
    print(f'    Peak effect:          {vals[peak_idx]:.6f} at h={peak_idx}')
    print(f'    Long-run (h=20):      {vals[-1]:.6f}')

### 6.3 Full IRF Grid

In [ ]:
# Full K x K IRF grid
fig = plot_irf_grid(irf, save_path='../outputs/figures/irfs/07_irf_full_grid.png')
plt.show()

### 6.4 Cumulative IRFs

Since GDP growth is a flow variable (rate of change), the cumulative IRF captures the total effect on the GDP **level**.

In [ ]:
# Compute cumulative IRFs
irf_cumulative = results.irf(
    periods=20,
    method='cholesky',
    order=ordering,
    cumulative=True
)

# Compare level vs cumulative for GDP growth
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

vals_level = irf['gdp_growth', 'interest_rate']
axes[0].plot(h, vals_level, color='#2166ac', linewidth=2, marker='o', markersize=3)
axes[0].fill_between(h, 0, vals_level, alpha=0.15, color='#2166ac')
axes[0].axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
axes[0].set_title('Level IRF (period-by-period)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Horizon (quarters)', fontsize=11)
axes[0].set_ylabel('Response of GDP Growth', fontsize=11)
axes[0].grid(True, alpha=0.3)

vals_cum = irf_cumulative['gdp_growth', 'interest_rate']
axes[1].plot(h, vals_cum, color='#b2182b', linewidth=2, marker='s', markersize=3)
axes[1].fill_between(h, 0, vals_cum, alpha=0.15, color='#b2182b')
axes[1].axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
axes[1].set_title('Cumulative IRF (effect on GDP level)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Horizon (quarters)', fontsize=11)
axes[1].set_ylabel('Cumulative Response', fontsize=11)
axes[1].grid(True, alpha=0.3)

fig.suptitle('GDP Growth Response to Monetary Shock: Level vs Cumulative',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_level_vs_cumulative.png',
            dpi=150, bbox_inches='tight')
plt.show()

print('Dynamic Multipliers (Cumulative IRF of GDP Growth):')
print(f'  Short-run (h=4):  {vals_cum[4]:.6f}')
print(f'  Medium-run (h=8): {vals_cum[8]:.6f}')
print(f'  Long-run (h=20):  {vals_cum[-1]:.6f}')

---

## Section 7: Variance Decomposition (25 min)

FEVD tells us what fraction of the forecast error variance in each variable is attributable to shocks in other variables.

### 7.1 Compute FEVD

In [ ]:
# Compute Cholesky FEVD
fevd = results.fevd(periods=20, method='cholesky', order=ordering)

print(f'FEVD computed successfully.')
print(f'  Method: {fevd.method}')
print(f'  Decomposition shape: {fevd.decomposition.shape}')
print(f'  Variables: {fevd.var_names}')
print()
print(fevd.summary(horizons=[1, 4, 8, 12, 20]))

In [ ]:
# Detailed FEVD for GDP growth
gdp_idx = list(fevd.var_names).index('gdp_growth')
ir_idx_fevd = list(fevd.var_names).index('interest_rate')

print('Variance Decomposition of GDP Growth')
print('=' * 80)
print(f'{"Horizon":>8s}', end='')
for var in fevd.var_names:
    print(f'{var:>18s}', end='')
print(f'{"Sum":>8s}')
print('-' * 80)

for horizon in [1, 4, 8, 12, 20]:
    print(f'{horizon:>8d}', end='')
    row_sum = 0
    for j in range(fevd.K):
        val = fevd.decomposition[horizon, gdp_idx, j]
        row_sum += val
        print(f'{val*100:>17.2f}%', end='')
    print(f'{row_sum*100:>7.1f}%')

print(f'\nMonetary policy (interest rate) explains '
      f'{fevd.decomposition[20, gdp_idx, ir_idx_fevd]*100:.1f}% of GDP variance at h=20.')

In [ ]:
# Stacked area charts for all variables
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

colors_fevd = sns.color_palette('husl', n_colors=fevd.K)
horizons_plot = np.arange(fevd.periods + 1)

for idx, var in enumerate(list(fevd.var_names)):
    ax = axes[idx // 2, idx % 2]
    var_idx_local = list(fevd.var_names).index(var)
    decomp = fevd.decomposition[:, var_idx_local, :]
    
    ax.stackplot(
        horizons_plot,
        *[decomp[:, j] for j in range(fevd.K)],
        labels=list(fevd.var_names),
        colors=colors_fevd,
        alpha=0.85
    )
    ax.set_xlabel('Horizon', fontsize=10)
    ax.set_ylabel('Share', fontsize=10)
    ax.set_title(f'FEVD: {var}', fontsize=12, fontweight='bold')
    ax.set_xlim(0, fevd.periods)
    ax.set_ylim(0, 1.0)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    ax.grid(True, alpha=0.3, axis='y')
    if idx == 1:
        ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5),
                  title='Shock', fontsize=9, title_fontsize=10)

fig.suptitle('Forecast Error Variance Decomposition (Cholesky)',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig('../outputs/figures/fevds/07_fevd_all.png',
            dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Built-in FEVD plot for GDP growth
fig = fevd.plot(variables=['gdp_growth'], backend='matplotlib', show=False)
if fig is not None:
    fig.savefig('../outputs/figures/fevds/07_fevd_gdp_builtin.png',
                dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Policy dominance summary
print('Share of Variance Explained by Monetary Shocks at h=20')
print('=' * 55)
for i, var in enumerate(fevd.var_names):
    share = fevd.decomposition[20, i, ir_idx_fevd] * 100
    bar = '#' * int(share)
    print(f'  {var:>16s}: {share:5.1f}%  {bar}')

---

## Section 8: Heterogeneity Analysis (30 min)

Do monetary policy shocks transmit differently in advanced vs. emerging/transition economies? We estimate separate Panel VARs for each group.

### 8.1 Define Country Groups

In [ ]:
# Define country groups
advanced = ['USA', 'DEU', 'JPN', 'GBR', 'FRA']
emerging = ['MEX', 'HUN', 'POL', 'CZE', 'KOR']

df_advanced = df[df['country'].isin(advanced)].copy()
df_emerging = df[df['country'].isin(emerging)].copy()

print(f'Advanced economies: {advanced}')
print(f'  Observations: {len(df_advanced)} ({df_advanced["country"].nunique()} countries)')
print(f'\nEmerging/transition economies: {emerging}')
print(f'  Observations: {len(df_emerging)} ({df_emerging["country"].nunique()} countries)')

print('\n\nVariable Means by Group:')
print('=' * 65)
print(f'{"Variable":>16s} {"Advanced":>12s} {"Emerging":>12s} {"Difference":>12s}')
print('-' * 65)
for var in variables:
    adv_mean = df_advanced[var].mean()
    emg_mean = df_emerging[var].mean()
    print(f'{var:>16s} {adv_mean:>12.3f} {emg_mean:>12.3f} {emg_mean - adv_mean:>12.3f}')

In [ ]:
# Estimate separate VARs
data_adv = PanelVARData(df_advanced, endog_vars=endog_vars,
    entity_col='country', time_col='quarter', lags=2)
model_adv = PanelVAR(data_adv)
results_adv = model_adv.fit(method='ols', cov_type='driscoll_kraay')

print('Advanced Economies VAR(2):')
print(f'  N={results_adv.N}, n_obs={results_adv.n_obs}')
print(f'  Stable: {results_adv.is_stable()}')
print(f'  Max eigenvalue modulus: {results_adv.max_eigenvalue_modulus:.4f}')

data_emg = PanelVARData(df_emerging, endog_vars=endog_vars,
    entity_col='country', time_col='quarter', lags=2)
model_emg = PanelVAR(data_emg)
results_emg = model_emg.fit(method='ols', cov_type='driscoll_kraay')

print(f'\nEmerging Economies VAR(2):')
print(f'  N={results_emg.N}, n_obs={results_emg.n_obs}')
print(f'  Stable: {results_emg.is_stable()}')
print(f'  Max eigenvalue modulus: {results_emg.max_eigenvalue_modulus:.4f}')

In [ ]:
# Compute IRFs for both groups
irf_adv = results_adv.irf(
    periods=20, method='cholesky', order=ordering,
    ci_method='bootstrap', n_bootstrap=500, ci_level=0.95,
    seed=42, verbose=False
)
irf_emg = results_emg.irf(
    periods=20, method='cholesky', order=ordering,
    ci_method='bootstrap', n_bootstrap=500, ci_level=0.95,
    seed=42, verbose=False
)
print('IRFs computed for both groups (B=500 bootstrap replications).')

In [ ]:
# Compare IRFs: Advanced vs Emerging
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
h = np.arange(irf_adv.periods + 1)

for idx, var in enumerate(response_vars):
    ax = axes[idx // 2, idx % 2]
    
    # Advanced
    vals_adv = irf_adv[var, 'interest_rate']
    r_adv = list(irf_adv.var_names).index(var)
    i_adv = list(irf_adv.var_names).index('interest_rate')
    if irf_adv.ci_lower is not None:
        ax.fill_between(h, irf_adv.ci_lower[:, r_adv, i_adv],
                        irf_adv.ci_upper[:, r_adv, i_adv],
                        alpha=0.15, color='#2166ac')
    ax.plot(h, vals_adv, color='#2166ac', linewidth=2, label='Advanced', linestyle='-')
    
    # Emerging
    vals_emg = irf_emg[var, 'interest_rate']
    r_emg = list(irf_emg.var_names).index(var)
    i_emg = list(irf_emg.var_names).index('interest_rate')
    if irf_emg.ci_lower is not None:
        ax.fill_between(h, irf_emg.ci_lower[:, r_emg, i_emg],
                        irf_emg.ci_upper[:, r_emg, i_emg],
                        alpha=0.15, color='#b2182b')
    ax.plot(h, vals_emg, color='#b2182b', linewidth=2, label='Emerging', linestyle='--')
    
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'Response: {var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=10)
    ax.set_ylabel('Response', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

fig.suptitle('Monetary Shock Transmission: Advanced vs Emerging Economies\n'
             '(Cholesky IRFs with 95% Bootstrap CI)',
             fontsize=14, fontweight='bold', y=1.03)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_heterogeneity_comparison.png',
            dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Quantitative comparison at key horizons
print('Heterogeneity in Monetary Transmission')
print('=' * 85)
print(f'{"Variable":>16s} {"Group":>12s} {"h=0":>10s} {"h=4":>10s} '
      f'{"h=8":>10s} {"h=12":>10s} {"h=20":>10s}')
print('-' * 85)

for var in ['inflation', 'gdp_growth', 'unemployment']:
    for group_name, irf_obj in [('Advanced', irf_adv), ('Emerging', irf_emg)]:
        vals = irf_obj[var, 'interest_rate']
        print(f'{var:>16s} {group_name:>12s}', end='')
        for horizon in [0, 4, 8, 12, 20]:
            print(f'{vals[horizon]:>10.4f}', end='')
        print()
    print()

print('Larger responses in emerging economies may reflect less anchored expectations.')
print('Faster convergence in advanced economies suggests more credible monetary policy.')

---

## Section 9: Robustness Checks (20 min)

### 9.1 Alternative Cholesky Orderings

In [ ]:
# Three alternative orderings
alternative_orderings = {
    'Baseline': ['interest_rate', 'inflation', 'gdp_growth', 'unemployment'],
    'Alt 1 (output first)': ['gdp_growth', 'inflation', 'interest_rate', 'unemployment'],
    'Alt 2 (policy last)': ['gdp_growth', 'unemployment', 'inflation', 'interest_rate'],
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
colors_ord = ['#2166ac', '#b2182b', '#4dac26']
styles = ['-', '--', '-.']
h = np.arange(21)

for resp_idx, resp_var in enumerate(['inflation', 'gdp_growth', 'unemployment']):
    ax = axes[resp_idx]
    for ord_idx, (label, ord_list) in enumerate(alternative_orderings.items()):
        irf_alt = results.irf(periods=20, method='cholesky', order=ord_list)
        vals = irf_alt[resp_var, 'interest_rate']
        ax.plot(h, vals, color=colors_ord[ord_idx], linewidth=2,
                linestyle=styles[ord_idx], label=label)
    
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'{resp_var} response', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=10)
    ax.set_ylabel('Response', fontsize=10)
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.3)

fig.suptitle('Robustness: IRFs across Alternative Cholesky Orderings\n'
             '(Response to Interest Rate Shock)',
             fontsize=14, fontweight='bold', y=1.04)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_ordering_robustness.png',
            dpi=150, bbox_inches='tight')
plt.show()

print('If IRFs are similar across orderings, the identification is robust.')
print('Large differences suggest sensitivity to the recursive assumption.')

### 9.2 Generalized IRFs (Order-Invariant)

In [ ]:
# Generalized IRFs (Pesaran-Shin, order-invariant)
irf_generalized = results.irf(periods=20, method='generalized')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
h = np.arange(21)

for idx, var in enumerate(['inflation', 'gdp_growth', 'unemployment']):
    ax = axes[idx]
    chol_vals = irf[var, 'interest_rate']
    gen_vals = irf_generalized[var, 'interest_rate']
    
    ax.plot(h, chol_vals, color='#2166ac', linewidth=2, label='Cholesky', linestyle='-')
    ax.plot(h, gen_vals, color='#b2182b', linewidth=2, label='Generalized', linestyle='--')
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'{var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon', fontsize=10)
    ax.set_ylabel('Response', fontsize=10)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

fig.suptitle('Cholesky vs Generalized IRFs: Response to Interest Rate Shock',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_cholesky_vs_generalized.png',
            dpi=150, bbox_inches='tight')
plt.show()

### 9.3 Sensitivity to Lag Order

In [ ]:
# Compare IRFs across lag orders p=1, 2, 3, 4
lag_orders = [1, 2, 3, 4]
lag_colors = ['#1b9e77', '#d95f02', '#7570b3', '#e7298a']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
h = np.arange(21)

for resp_idx, resp_var in enumerate(['inflation', 'gdp_growth', 'unemployment']):
    ax = axes[resp_idx]
    for lag_idx, p_lags in enumerate(lag_orders):
        try:
            data_p = PanelVARData(df, endog_vars=endog_vars, entity_col='country',
                                 time_col='quarter', lags=p_lags)
            model_p = PanelVAR(data_p)
            results_p = model_p.fit(method='ols', cov_type='driscoll_kraay')
            irf_p = results_p.irf(periods=20, method='cholesky', order=ordering)
            vals = irf_p[resp_var, 'interest_rate']
            stable_label = 'S' if results_p.is_stable() else 'U'
            ax.plot(h, vals, color=lag_colors[lag_idx], linewidth=2,
                    label=f'p={p_lags} [{stable_label}]')
        except Exception as e:
            print(f'  Lag {p_lags}: Error - {e}')
    
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'{resp_var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon', fontsize=10)
    ax.set_ylabel('Response', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

fig.suptitle('Robustness: IRFs across Lag Orders (p=1,2,3,4)\n'
             'S=Stable, U=Unstable',
             fontsize=14, fontweight='bold', y=1.04)
fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_lag_robustness.png',
            dpi=150, bbox_inches='tight')
plt.show()

print('If results are qualitatively similar across lag orders, findings are robust.')

---

## Section 10: Summary and Conclusions (20 min)

### Key Findings

In [ ]:
# Automated summary
print('=' * 70)
print('SUMMARY OF KEY FINDINGS')
print('=' * 70)

# 1. Monetary Policy Effectiveness
print('\n1. MONETARY POLICY EFFECTIVENESS')
print('-' * 40)
gdp_resp = irf['gdp_growth', 'interest_rate']
gdp_peak_h = np.argmax(np.abs(gdp_resp))
print(f'   GDP growth peak response: {gdp_resp[gdp_peak_h]:.4f} at h={gdp_peak_h}')
print(f'   GDP growth impact (h=0):  {gdp_resp[0]:.4f}')
unemp_resp = irf['unemployment', 'interest_rate']
unemp_peak_h = np.argmax(np.abs(unemp_resp))
print(f'   Unemployment peak:        {unemp_resp[unemp_peak_h]:.4f} at h={unemp_peak_h}')

# 2. Transmission Timing
print('\n2. TRANSMISSION TIMING')
print('-' * 40)
for var in ['inflation', 'gdp_growth', 'unemployment']:
    vals = irf[var, 'interest_rate']
    peak_h = np.argmax(np.abs(vals))
    peak_val = vals[peak_h]
    half_target = peak_val / 2
    half_life = peak_h
    for hh in range(peak_h, len(vals)):
        if abs(vals[hh]) <= abs(half_target):
            half_life = hh
            break
    print(f'   {var:>16s}: peak at h={peak_h}, half-life ~h={half_life}')

# 3. Variance Decomposition
print('\n3. VARIANCE DECOMPOSITION (h=20)')
print('-' * 40)
for i, var in enumerate(fevd.var_names):
    ir_share = fevd.decomposition[20, i, ir_idx_fevd] * 100
    print(f'   {var:>16s}: {ir_share:5.1f}% from monetary shocks')

# 4. Heterogeneity
print('\n4. HETEROGENEITY: ADVANCED vs EMERGING')
print('-' * 40)
for var in ['gdp_growth', 'inflation']:
    adv_peak = np.max(np.abs(irf_adv[var, 'interest_rate']))
    emg_peak = np.max(np.abs(irf_emg[var, 'interest_rate']))
    ratio = emg_peak / adv_peak if adv_peak > 0 else float('inf')
    print(f'   {var:>16s}: Adv peak={adv_peak:.4f}, Emg peak={emg_peak:.4f} (ratio={ratio:.2f})')

# 5. Granger Causality
print('\n5. GRANGER CAUSALITY HIGHLIGHTS')
print('-' * 40)
for _, row in pw_df.head(5).iterrows():
    stars = '***' if row['p_value'] < 0.01 else '**' if row['p_value'] < 0.05 else '*' if row['p_value'] < 0.10 else ''
    print(f'   {row["Cause"]:>16s} -> {row["Effect"]:<16s} p={row["p_value"]:.4f} {stars}')

print('\n' + '=' * 70)

### Economic Interpretation

**Monetary Policy Effectiveness:**
A contractionary monetary shock (interest rate increase) leads to a decline in GDP growth and an increase in unemployment, consistent with standard macroeconomic theory (IS curve). The peak effect on GDP typically occurs with a lag of several quarters, reflecting the well-known "long and variable lags" of monetary policy (Friedman, 1961).

**Inflation Response:**
The inflation response to a monetary tightening may exhibit a "price puzzle" (initial increase) in the short run, a common finding in VAR-based monetary policy analysis (Sims, 1992). The contractionary effect on inflation becomes evident at longer horizons.

**Transmission Heterogeneity:**
Advanced and emerging economies show different magnitudes and timing in their responses, reflecting differences in:
- Financial market development and the interest rate channel
- Exchange rate regimes and pass-through
- Central bank credibility and expectations anchoring
- Structural rigidities in labor and product markets

**Policy Implications:**
1. Central banks should account for the delayed effects of monetary policy when setting rates
2. The heterogeneity across country groups suggests that "one size fits all" monetary policy rules are inappropriate
3. Variance decomposition reveals the relative importance of monetary vs. supply shocks for output stabilization

---

## Section 11: Export Results (10 min)

In [ ]:
# Save Granger causality matrix
gc_matrix.to_csv('../outputs/tables/07_granger_causality_matrix.csv')
print('Saved: ../outputs/tables/07_granger_causality_matrix.csv')

# Save pairwise Granger results
pw_df.to_csv('../outputs/tables/07_pairwise_granger.csv', index=False)
print('Saved: ../outputs/tables/07_pairwise_granger.csv')

# Save IRF at key horizons
irf_records = []
for var in response_vars:
    for horizon in [0, 1, 2, 4, 8, 12, 16, 20]:
        irf_records.append({
            'response': var, 'impulse': 'interest_rate',
            'horizon': horizon,
            'irf_value': irf[var, 'interest_rate'][horizon],
        })
irf_export_df = pd.DataFrame(irf_records)
irf_export_df.to_csv('../outputs/tables/07_irf_monetary_shock.csv', index=False)
print('Saved: ../outputs/tables/07_irf_monetary_shock.csv')

# Save FEVD at key horizons
fevd_records = []
for i, var in enumerate(fevd.var_names):
    for horizon in [1, 4, 8, 12, 20]:
        for j, shock_var in enumerate(fevd.var_names):
            fevd_records.append({
                'variable': var, 'shock': shock_var,
                'horizon': horizon,
                'share': fevd.decomposition[horizon, i, j],
            })
fevd_export_df = pd.DataFrame(fevd_records)
fevd_export_df.to_csv('../outputs/tables/07_fevd_all.csv', index=False)
print('Saved: ../outputs/tables/07_fevd_all.csv')

# Save heterogeneity comparison
hetero_records = []
for var in ['inflation', 'gdp_growth', 'unemployment']:
    for group_name, irf_obj in [('Advanced', irf_adv), ('Emerging', irf_emg)]:
        vals = irf_obj[var, 'interest_rate']
        for horizon in [0, 4, 8, 12, 20]:
            hetero_records.append({
                'variable': var, 'group': group_name,
                'horizon': horizon, 'irf_value': vals[horizon],
            })
hetero_df = pd.DataFrame(hetero_records)
hetero_df.to_csv('../outputs/tables/07_heterogeneity_irf.csv', index=False)
print('Saved: ../outputs/tables/07_heterogeneity_irf.csv')

print('\nAll results exported successfully.')

In [ ]:
# Generate HTML report
import html as html_module
from datetime import datetime

html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Monetary Policy Transmission Analysis</title>
    <style>
        body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; color: #333; }}
        h1 {{ color: #2166ac; border-bottom: 3px solid #2166ac; padding-bottom: 10px; }}
        h2 {{ color: #4393c3; margin-top: 30px; }}
        h3 {{ color: #666; }}
        table {{ border-collapse: collapse; width: 100%; margin: 15px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: right; }}
        th {{ background-color: #2166ac; color: white; }}
        tr:nth-child(even) {{ background-color: #f2f2f2; }}
        .summary-box {{ background: #f0f7ff; border: 1px solid #b2d3ea; border-radius: 8px; padding: 15px; margin: 15px 0; }}
        .finding {{ background: #fff8e1; border-left: 4px solid #ffc107; padding: 10px 15px; margin: 10px 0; }}
        .footer {{ text-align: center; color: #999; margin-top: 40px; font-size: 0.9em; }}
        img {{ max-width: 100%; height: auto; margin: 10px 0; border: 1px solid #eee; }}
    </style>
</head>
<body>
<h1>Monetary Policy Transmission Analysis</h1>
<p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>
<p><strong>Data:</strong> {results.N} countries, {df['quarter'].nunique()} quarters ({df['quarter'].min()} to {df['quarter'].max()})</p>
<p><strong>Model:</strong> Panel VAR({results.p}) with Driscoll-Kraay standard errors</p>

<div class="summary-box">
<h3>Executive Summary</h3>
<p>This report analyzes the transmission of monetary policy shocks across {results.N} OECD countries
using a Panel VAR framework. Key findings indicate that contractionary monetary policy (interest rate increases)
significantly reduces GDP growth and inflation, with peak effects occurring several quarters after the shock.</p>
</div>

<h2>1. Model Specification</h2>
<table>
<tr><th>Parameter</th><th>Value</th></tr>
<tr><td>Endogenous variables</td><td>{', '.join(endog_vars)}</td></tr>
<tr><td>Number of lags (p)</td><td>{results.p}</td></tr>
<tr><td>Number of countries (N)</td><td>{results.N}</td></tr>
<tr><td>Observations</td><td>{results.n_obs}</td></tr>
<tr><td>Covariance type</td><td>Driscoll-Kraay</td></tr>
<tr><td>Stable</td><td>{results.is_stable()}</td></tr>
<tr><td>Max eigenvalue modulus</td><td>{results.max_eigenvalue_modulus:.6f}</td></tr>
<tr><td>AIC</td><td>{results.aic:.4f}</td></tr>
<tr><td>BIC</td><td>{results.bic:.4f}</td></tr>
</table>

<h2>2. Granger Causality Results</h2>
<p>The table below shows p-values for pairwise Granger causality tests. Values below 0.05 indicate
statistically significant predictive relationships.</p>
<table>
<tr><th>Cause / Effect</th>"""

# Add Granger causality table
for col in gc_matrix.columns:
    html_content += f"<th>{html_module.escape(col)}</th>"
html_content += "</tr>"
for idx_row, row in gc_matrix.iterrows():
    html_content += f"<tr><td><strong>{html_module.escape(str(idx_row))}</strong></td>"
    for val in row:
        if pd.notna(val):
            color = '#d4edda' if val < 0.05 else '#fff'
            html_content += f'<td style="background:{color}">{val:.4f}</td>'
        else:
            html_content += '<td>--</td>'
    html_content += "</tr>"
html_content += "</table>"

# IRF key results
html_content += """
<h2>3. Impulse Response Functions</h2>
<p>Responses to a one-standard-deviation contractionary monetary shock (Cholesky identification):</p>
<table>
<tr><th>Response Variable</th><th>Impact (h=0)</th><th>h=4</th><th>h=8</th><th>Peak Effect</th><th>Peak Horizon</th></tr>
"""
for var in ['inflation', 'gdp_growth', 'unemployment']:
    vals = irf[var, 'interest_rate']
    peak_h = np.argmax(np.abs(vals))
    html_content += f"""<tr>
    <td><strong>{var}</strong></td>
    <td>{vals[0]:.6f}</td><td>{vals[4]:.6f}</td><td>{vals[8]:.6f}</td>
    <td>{vals[peak_h]:.6f}</td><td>h={peak_h}</td>
    </tr>"""
html_content += "</table>"

# FEVD results
html_content += """
<h2>4. Variance Decomposition</h2>
<p>Share of forecast error variance explained by monetary shocks (interest rate) at selected horizons:</p>
<table>
<tr><th>Variable</th><th>h=1</th><th>h=4</th><th>h=8</th><th>h=20</th></tr>
"""
for i, var in enumerate(fevd.var_names):
    html_content += f"<tr><td><strong>{var}</strong></td>"
    for horizon in [1, 4, 8, 20]:
        share = fevd.decomposition[horizon, i, ir_idx_fevd] * 100
        html_content += f"<td>{share:.1f}%</td>"
    html_content += "</tr>"
html_content += "</table>"

# Heterogeneity results
html_content += """
<h2>5. Heterogeneity: Advanced vs Emerging Economies</h2>
<div class="finding">
<p><strong>Finding:</strong> Advanced and emerging economies show different transmission patterns,
reflecting differences in financial market development, central bank credibility, and structural rigidities.</p>
</div>
<table>
<tr><th>Variable</th><th>Group</th><th>Peak Effect</th><th>Peak Horizon</th></tr>
"""
for var in ['gdp_growth', 'inflation']:
    for group_name, irf_obj in [('Advanced', irf_adv), ('Emerging', irf_emg)]:
        vals = irf_obj[var, 'interest_rate']
        peak_h = np.argmax(np.abs(vals))
        html_content += f"""<tr><td>{var}</td><td>{group_name}</td>
        <td>{vals[peak_h]:.6f}</td><td>h={peak_h}</td></tr>"""
html_content += "</table>"

# Conclusions
html_content += """
<h2>6. Policy Implications</h2>
<div class="finding">
<p>1. <strong>Monetary policy is effective</strong> but operates with significant lags.</p>
<p>2. <strong>Forward-looking policy</strong> is essential given the delayed transmission.</p>
<p>3. <strong>Heterogeneity</strong> across country groups argues against uniform policy rules.</p>
<p>4. <strong>Communication and credibility</strong> amplify the effectiveness of interest rate adjustments.</p>
</div>

<div class="footer">
<p>Generated by PanelBox VAR Analysis | Panel VAR Tutorial Series</p>
</div>
</body>
</html>"""

# Save the report
report_path = '../outputs/monetary_policy_report.html'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f'HTML report generated: {report_path}')
print(f'Report size: {len(html_content):,} characters')

### Exercise 1: Alternative Specifications (Easy)

Include an additional variable (e.g., exchange rate or money supply) in the VAR and re-run the full analysis. Also test robustness by excluding the crisis period (2008-2009) and comparing IRFs.

**Tasks:**
1. Re-estimate the VAR with only 3 variables (drop unemployment): `['interest_rate', 'inflation', 'gdp_growth']`
2. Exclude the crisis period (2008-2009 quarters) from the data and re-estimate the baseline 4-variable model
3. Compare the inflation response to a monetary shock across specifications

**Expected output:**
- IRF comparison plot (full sample vs crisis-excluded) for the inflation response to monetary shock
- Table of key IRF peak magnitudes across specifications
- Discussion of whether monetary policy effects are crisis-dependent

In [ ]:
# YOUR CODE HERE

### Exercise 2: Country-by-Country Heterogeneity (Medium)

Estimate individual VARs for each country (or a subset of 5-6 countries). Compare the inflation IRF to a monetary policy shock across countries. Test for structural breaks using subsample analysis (pre/post 2008).

**Tasks:**
1. Estimate separate time-series VARs for 5 countries: USA, DEU, JPN, MEX, KOR
2. Compute the inflation response to a monetary shock for each country
3. Calculate the half-life of adjustment for each country
4. Create a bar chart of half-life of inflation adjustment by country

**Expected output:**
- Multi-panel figure showing country-specific inflation IRFs overlaid on the panel IRF
- Table of peak response magnitudes and timing by country
- Bar chart of half-life of inflation adjustment by country

In [ ]:
# YOUR CODE HERE

### Exercise 3: GMM Extension for Short Panels (Hard)

Subset the data to T=15 quarters (simulating a short panel scenario). Compare OLS-based VAR results with the theoretical Nickell bias. Run full diagnostic analysis.

**Tasks:**
1. Subset the data to the last 15 quarters only
2. Estimate the VAR using OLS on the short panel
3. Compare coefficient estimates from the full sample (T=80) vs the short panel (T=15)
4. Calculate the theoretical Nickell bias for T=15 and discuss its implications
5. Discuss when GMM (Arellano-Bond) would be necessary vs when OLS is acceptable

**Expected output:**
- Side-by-side comparison table: Full sample vs Short panel coefficient estimates and standard errors
- Diagnostic summary (stability, information criteria comparison)
- IRF comparison plot across estimation samples
- Theoretical Nickell bias chart as function of T

In [ ]:
# YOUR CODE HERE

### Exercise 4: Policy Brief (Medium)

Write a structured 2-page policy brief summarizing the main findings. Include: (a) key stylized facts from EDA, (b) main IRF results with confidence intervals, (c) FEVD decomposition at policy-relevant horizons, (d) heterogeneity insights, and (e) policy recommendations.

**Tasks:**
1. Compile key statistics from the analysis
2. Create at least 2 publication-quality figures
3. Write an executive summary, methodology section, key findings, and policy implications
4. Format as a structured markdown cell

**Expected output:**
- Markdown cell with formatted policy brief
- At least 2 publication-quality figures embedded
- Executive summary, methodology section, key findings, and policy implications clearly structured

In [ ]:
# YOUR CODE HERE

---

## References

- Christiano, L. J., Eichenbaum, M., & Evans, C. L. (1999). Monetary policy shocks: What have we learned and to what end? *Handbook of Macroeconomics*, 1, 65–148.
- Dumitrescu, E. I., & Hurlin, C. (2012). Testing for Granger non-causality in heterogeneous panels. *Economic Modelling*, 29(4), 1450–1460.
- Friedman, M. (1961). The lag in effect of monetary policy. *Journal of Political Economy*, 69(5), 447–466.
- Granger, C. W. J. (1969). Investigating causal relations by econometric models and cross-spectral methods. *Econometrica*, 37(3), 424–438.
- Holtz-Eakin, D., Newey, W., & Rosen, H. S. (1988). Estimating vector autoregressions with panel data. *Econometrica*, 56(6), 1371–1395.
- Lutkepohl, H. (2005). *New Introduction to Multiple Time Series Analysis*. Springer.
- Pesaran, H. H., & Shin, Y. (1998). Generalized impulse response analysis in linear multivariate models. *Economics Letters*, 58(1), 17–29.
- Romer, C. D., & Romer, D. H. (2004). A new measure of monetary shocks. *American Economic Review*, 94(4), 1055–1084.
- Sims, C. A. (1980). Macroeconomics and reality. *Econometrica*, 48(1), 1–48.
- Sims, C. A. (1992). Interpreting the macroeconomic time series facts: The effects of monetary policy. *European Economic Review*, 36(5), 975–1000.
- Taylor, J. B. (1993). Discretion versus policy rules in practice. *Carnegie-Rochester Conference Series on Public Policy*, 39, 195–214.